In [1]:
import re
import numpy as np
import nltk
import collections
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

# BASIC IMPLEMENTATION

TF-IDF

In [2]:
def get_tfidf(sentences,stop_words):
    corpus = []
    for sent in sentences:
        sent_token = nltk.word_tokenize(sent)
        sent_list = [word for word in sent_token if word not in stop_words]
        sent_str = ' '.join(sent_list)
        corpus.append(sent_str)

    vectorizer = CountVectorizer()
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))
    return tfidf.toarray()

### Algorithms to Evaluate the Importance of Sentences

- The frequency of words

In [3]:
def weight_frequency(tfidf):
    weight = {}
    for i in range(len(tfidf)):
        weight[i] = np.sum(tfidf[i])

    # Normalization
    max_weight = max(weight.values())
    min_weight = min(weight.values())
    for key in weight.keys():
        x = weight[key]
        weight[key] = (x-min_weight)/(max_weight-min_weight)

    return weight

- Position in document

In [4]:
def weight_position(sentences):
    weight = {}
    n = len(sentences)
    for i in range(n):
        weight[i] = (n - i)/n
    return weight

- The similarity of sentences

In [5]:
def get_similarity(s1, s2):
    return np.sum(s1*s2)/(1e-6+(np.sqrt(np.sum(s1*s1))*np.sqrt(np.sum(s2*s2))))

def weight_similarity(tfidf):
    weight = collections.defaultdict(lambda :0)
    for i in range(len(tfidf)):
        score_i = 0
        for j in range(len(tfidf)):
            score_i += get_similarity(tfidf[i], tfidf[j])
        weight[i] = score_i

    # Normalization
    max_score = max(weight.values())
    min_score = min(weight.values())
    for k in weight.keys():
        x = weight[k]
        weight[k] = (x-min_score)/(max_score-min_score)

    return weight

Compute the composite score, then sort it in descending order (default: setting all of the weights as even).

In [6]:
def final_score(frequency, position, similarity, feature_weight = [1,1,1]):
    weight = collections.defaultdict(lambda :0)
    for k in frequency.keys():
        weight[k] = feature_weight[0]*frequency[k]+feature_weight[1]*position[k]+feature_weight[2]*similarity[k]

    return sorted(weight.items(), key = lambda x:x[1], reverse = True)

In [7]:
def get_summarization(text, stop_word, num_sentence, comp_ratio = 0.3):
    sentences = re.sub(r'\n',' ',text) 
    sentences = nltk.sent_tokenize(text)
    
    tfidf = get_tfidf(sentences, stop_word)
    frequency = weight_frequency(tfidf)
    position = weight_position(sentences)
    similarity = weight_similarity(tfidf)
    
    score = final_score(frequency, position, similarity, feature_weight = [1,1,1])
    
    if num_sentence is None:
        num_sentence = int(len(sentences)*comp_ratio)
    sent_id = sorted([sent[0] for sent in score[:num_sentence]])
    
    summary = []
    for id in sent_id:
        summary.append(sentences[id])

    return ' '.join(summary)

### EVALUATION

In [8]:
import os

article_dir = "./my_evalue/articles"
result_dir = "./my_evalue/basic"

stop_word = nltk.corpus.stopwords.words('english')

for id in range(11490):
    id = str(id)
    article_name = str(id).zfill(6) + "_article.txt"
    result_name = str(id).zfill(6) + "_result.txt"
    article_path = os.path.join(article_dir, article_name)
    result_path = os.path.join(result_dir, result_name)
    
    with open(article_path, "r") as f:
        article_doc = f.read().splitlines()
        current_article = " ".join(article_doc)
    
    result = get_summarization(current_article, stop_word, num_sentence = 3)
    
    with open(result_path,"w") as f:
        f.write(result)
        if int(id)%500==0:
            print("Write example ",result_name, ".")

Write example  000000_result.txt .
Write example  000500_result.txt .
Write example  001000_result.txt .
Write example  001500_result.txt .
Write example  002000_result.txt .
Write example  002500_result.txt .
Write example  003000_result.txt .
Write example  003500_result.txt .
Write example  004000_result.txt .
Write example  004500_result.txt .
Write example  005000_result.txt .
Write example  005500_result.txt .
Write example  006000_result.txt .
Write example  006500_result.txt .
Write example  007000_result.txt .
Write example  007500_result.txt .
Write example  008000_result.txt .
Write example  008500_result.txt .
Write example  009000_result.txt .
Write example  009500_result.txt .
Write example  010000_result.txt .
Write example  010500_result.txt .
Write example  011000_result.txt .


In [1]:
from pyrouge import Rouge155

r = Rouge155()
r.system_dir = "./my_evalue/reference"
r.model_dir = "./my_evalue/basic"
r.system_filename_pattern = '(\d+)_reference.txt'
r.model_filename_pattern = '#ID#_result.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2021-06-30 14:55:57,676 [MainThread  ] [INFO ]  Writing summaries.
2021-06-30 14:55:57,687 [MainThread  ] [INFO ]  Processing summaries. Saving system files to C:\Users\KELLYL~1\AppData\Local\Temp\tmp2b8v6ix_\system and model files to C:\Users\KELLYL~1\AppData\Local\Temp\tmp2b8v6ix_\model.
2021-06-30 14:55:57,688 [MainThread  ] [INFO ]  Processing files in ./my_evalue/reference.
2021-06-30 14:55:58,342 [MainThread  ] [INFO ]  Processing 000000_reference.txt.
2021-06-30 14:55:58,345 [MainThread  ] [INFO ]  Processing 000001_reference.txt.
2021-06-30 14:55:58,348 [MainThread  ] [INFO ]  Processing 000002_reference.txt.
2021-06-30 14:55:58,373 [MainThread  ] [INFO ]  Processing 000003_reference.txt.
2021-06-30 14:55:58,377 [MainThread  ] [INFO ]  Processing 000004_reference.txt.
2021-06-30 14:55:58,380 [MainThread  ] [INFO ]  Processing 000005_reference.txt.
2021-06-30 14:55:58,385 [MainThread  ] [INFO ]  Processing 000006_reference.txt.
2021-06-30 14:55:58,389 [MainThread  ] [INFO ]  Pro

2021-06-30 14:55:58,698 [MainThread  ] [INFO ]  Processing 000097_reference.txt.
2021-06-30 14:55:58,703 [MainThread  ] [INFO ]  Processing 000098_reference.txt.
2021-06-30 14:55:58,707 [MainThread  ] [INFO ]  Processing 000099_reference.txt.
2021-06-30 14:55:58,710 [MainThread  ] [INFO ]  Processing 000100_reference.txt.
2021-06-30 14:55:58,713 [MainThread  ] [INFO ]  Processing 000101_reference.txt.
2021-06-30 14:55:58,716 [MainThread  ] [INFO ]  Processing 000102_reference.txt.
2021-06-30 14:55:58,721 [MainThread  ] [INFO ]  Processing 000103_reference.txt.
2021-06-30 14:55:58,724 [MainThread  ] [INFO ]  Processing 000104_reference.txt.
2021-06-30 14:55:58,727 [MainThread  ] [INFO ]  Processing 000105_reference.txt.
2021-06-30 14:55:58,730 [MainThread  ] [INFO ]  Processing 000106_reference.txt.
2021-06-30 14:55:58,734 [MainThread  ] [INFO ]  Processing 000107_reference.txt.
2021-06-30 14:55:58,736 [MainThread  ] [INFO ]  Processing 000108_reference.txt.
2021-06-30 14:55:58,739 [Mai

2021-06-30 14:55:59,028 [MainThread  ] [INFO ]  Processing 000199_reference.txt.
2021-06-30 14:55:59,032 [MainThread  ] [INFO ]  Processing 000200_reference.txt.
2021-06-30 14:55:59,035 [MainThread  ] [INFO ]  Processing 000201_reference.txt.
2021-06-30 14:55:59,038 [MainThread  ] [INFO ]  Processing 000202_reference.txt.
2021-06-30 14:55:59,041 [MainThread  ] [INFO ]  Processing 000203_reference.txt.
2021-06-30 14:55:59,044 [MainThread  ] [INFO ]  Processing 000204_reference.txt.
2021-06-30 14:55:59,048 [MainThread  ] [INFO ]  Processing 000205_reference.txt.
2021-06-30 14:55:59,051 [MainThread  ] [INFO ]  Processing 000206_reference.txt.
2021-06-30 14:55:59,054 [MainThread  ] [INFO ]  Processing 000207_reference.txt.
2021-06-30 14:55:59,057 [MainThread  ] [INFO ]  Processing 000208_reference.txt.
2021-06-30 14:55:59,060 [MainThread  ] [INFO ]  Processing 000209_reference.txt.
2021-06-30 14:55:59,063 [MainThread  ] [INFO ]  Processing 000210_reference.txt.
2021-06-30 14:55:59,067 [Mai

2021-06-30 14:55:59,364 [MainThread  ] [INFO ]  Processing 000301_reference.txt.
2021-06-30 14:55:59,368 [MainThread  ] [INFO ]  Processing 000302_reference.txt.
2021-06-30 14:55:59,371 [MainThread  ] [INFO ]  Processing 000303_reference.txt.
2021-06-30 14:55:59,374 [MainThread  ] [INFO ]  Processing 000304_reference.txt.
2021-06-30 14:55:59,377 [MainThread  ] [INFO ]  Processing 000305_reference.txt.
2021-06-30 14:55:59,381 [MainThread  ] [INFO ]  Processing 000306_reference.txt.
2021-06-30 14:55:59,385 [MainThread  ] [INFO ]  Processing 000307_reference.txt.
2021-06-30 14:55:59,388 [MainThread  ] [INFO ]  Processing 000308_reference.txt.
2021-06-30 14:55:59,460 [MainThread  ] [INFO ]  Processing 000309_reference.txt.
2021-06-30 14:55:59,463 [MainThread  ] [INFO ]  Processing 000310_reference.txt.
2021-06-30 14:55:59,467 [MainThread  ] [INFO ]  Processing 000311_reference.txt.
2021-06-30 14:55:59,470 [MainThread  ] [INFO ]  Processing 000312_reference.txt.
2021-06-30 14:55:59,473 [Mai

2021-06-30 14:55:59,783 [MainThread  ] [INFO ]  Processing 000403_reference.txt.
2021-06-30 14:55:59,786 [MainThread  ] [INFO ]  Processing 000404_reference.txt.
2021-06-30 14:55:59,790 [MainThread  ] [INFO ]  Processing 000405_reference.txt.
2021-06-30 14:55:59,794 [MainThread  ] [INFO ]  Processing 000406_reference.txt.
2021-06-30 14:55:59,797 [MainThread  ] [INFO ]  Processing 000407_reference.txt.
2021-06-30 14:55:59,800 [MainThread  ] [INFO ]  Processing 000408_reference.txt.
2021-06-30 14:55:59,803 [MainThread  ] [INFO ]  Processing 000409_reference.txt.
2021-06-30 14:55:59,806 [MainThread  ] [INFO ]  Processing 000410_reference.txt.
2021-06-30 14:55:59,809 [MainThread  ] [INFO ]  Processing 000411_reference.txt.
2021-06-30 14:55:59,812 [MainThread  ] [INFO ]  Processing 000412_reference.txt.
2021-06-30 14:55:59,816 [MainThread  ] [INFO ]  Processing 000413_reference.txt.
2021-06-30 14:55:59,819 [MainThread  ] [INFO ]  Processing 000414_reference.txt.
2021-06-30 14:55:59,822 [Mai

...

2021-06-30 14:57:16,219 [MainThread  ] [INFO ]  Processing 011159_result.txt.
2021-06-30 14:57:16,222 [MainThread  ] [INFO ]  Processing 011160_result.txt.
2021-06-30 14:57:16,226 [MainThread  ] [INFO ]  Processing 011161_result.txt.
2021-06-30 14:57:16,229 [MainThread  ] [INFO ]  Processing 011162_result.txt.
2021-06-30 14:57:16,233 [MainThread  ] [INFO ]  Processing 011163_result.txt.
2021-06-30 14:57:16,236 [MainThread  ] [INFO ]  Processing 011164_result.txt.
2021-06-30 14:57:16,239 [MainThread  ] [INFO ]  Processing 011165_result.txt.
2021-06-30 14:57:16,241 [MainThread  ] [INFO ]  Processing 011166_result.txt.
2021-06-30 14:57:16,244 [MainThread  ] [INFO ]  Processing 011167_result.txt.
2021-06-30 14:57:16,247 [MainThread  ] [INFO ]  Processing 011168_result.txt.
2021-06-30 14:57:16,251 [MainThread  ] [INFO ]  Processing 011169_result.txt.
2021-06-30 14:57:16,255 [MainThread  ] [INFO ]  Processing 011170_result.txt.
2021-06-30 14:57:16,258 [MainThread  ] [INFO ]  Processing 01117

2021-06-30 14:57:16,582 [MainThread  ] [INFO ]  Processing 011265_result.txt.
2021-06-30 14:57:16,586 [MainThread  ] [INFO ]  Processing 011266_result.txt.
2021-06-30 14:57:16,589 [MainThread  ] [INFO ]  Processing 011267_result.txt.
2021-06-30 14:57:16,592 [MainThread  ] [INFO ]  Processing 011268_result.txt.
2021-06-30 14:57:16,596 [MainThread  ] [INFO ]  Processing 011269_result.txt.
2021-06-30 14:57:16,598 [MainThread  ] [INFO ]  Processing 011270_result.txt.
2021-06-30 14:57:16,602 [MainThread  ] [INFO ]  Processing 011271_result.txt.
2021-06-30 14:57:16,605 [MainThread  ] [INFO ]  Processing 011272_result.txt.
2021-06-30 14:57:16,608 [MainThread  ] [INFO ]  Processing 011273_result.txt.
2021-06-30 14:57:16,612 [MainThread  ] [INFO ]  Processing 011274_result.txt.
2021-06-30 14:57:16,615 [MainThread  ] [INFO ]  Processing 011275_result.txt.
2021-06-30 14:57:16,619 [MainThread  ] [INFO ]  Processing 011276_result.txt.
2021-06-30 14:57:16,623 [MainThread  ] [INFO ]  Processing 01127

2021-06-30 14:57:16,938 [MainThread  ] [INFO ]  Processing 011371_result.txt.
2021-06-30 14:57:16,941 [MainThread  ] [INFO ]  Processing 011372_result.txt.
2021-06-30 14:57:16,945 [MainThread  ] [INFO ]  Processing 011373_result.txt.
2021-06-30 14:57:16,948 [MainThread  ] [INFO ]  Processing 011374_result.txt.
2021-06-30 14:57:16,951 [MainThread  ] [INFO ]  Processing 011375_result.txt.
2021-06-30 14:57:16,954 [MainThread  ] [INFO ]  Processing 011376_result.txt.
2021-06-30 14:57:16,958 [MainThread  ] [INFO ]  Processing 011377_result.txt.
2021-06-30 14:57:16,961 [MainThread  ] [INFO ]  Processing 011378_result.txt.
2021-06-30 14:57:16,964 [MainThread  ] [INFO ]  Processing 011379_result.txt.
2021-06-30 14:57:16,967 [MainThread  ] [INFO ]  Processing 011380_result.txt.
2021-06-30 14:57:16,971 [MainThread  ] [INFO ]  Processing 011381_result.txt.
2021-06-30 14:57:16,975 [MainThread  ] [INFO ]  Processing 011382_result.txt.
2021-06-30 14:57:16,978 [MainThread  ] [INFO ]  Processing 01138

2021-06-30 14:57:17,291 [MainThread  ] [INFO ]  Processing 011477_result.txt.
2021-06-30 14:57:17,295 [MainThread  ] [INFO ]  Processing 011478_result.txt.
2021-06-30 14:57:17,298 [MainThread  ] [INFO ]  Processing 011479_result.txt.
2021-06-30 14:57:17,301 [MainThread  ] [INFO ]  Processing 011480_result.txt.
2021-06-30 14:57:17,305 [MainThread  ] [INFO ]  Processing 011481_result.txt.
2021-06-30 14:57:17,309 [MainThread  ] [INFO ]  Processing 011482_result.txt.
2021-06-30 14:57:17,312 [MainThread  ] [INFO ]  Processing 011483_result.txt.
2021-06-30 14:57:17,315 [MainThread  ] [INFO ]  Processing 011484_result.txt.
2021-06-30 14:57:17,318 [MainThread  ] [INFO ]  Processing 011485_result.txt.
2021-06-30 14:57:17,322 [MainThread  ] [INFO ]  Processing 011486_result.txt.
2021-06-30 14:57:17,325 [MainThread  ] [INFO ]  Processing 011487_result.txt.
2021-06-30 14:57:17,328 [MainThread  ] [INFO ]  Processing 011488_result.txt.
2021-06-30 14:57:17,331 [MainThread  ] [INFO ]  Processing 01148

CalledProcessError: Command '['perl', 'E:\\ROUGE-1.5.5\\RELEASE-1.5.5\\ROUGE-1.5.5.pl', '-e', 'E:\\ROUGE-1.5.5\\RELEASE-1.5.5\\data', '-c', '95', '-2', '-1', '-U', '-r', '1000', '-n', '4', '-w', '1.2', '-a', 'C:\\Users\\KELLYL~1\\AppData\\Local\\Temp\\tmp294vkh4w\\rouge_conf.xml']' returned non-zero exit status 1.

## Result by pyrouge:

---------------------------------------------  
1 ROUGE-1 Average_R: 0.31473 (95%-conf.int. 0.31261 - 0.31674)  
1 ROUGE-1 Average_P: 0.53473 (95%-conf.int. 0.53215 - 0.53747)  
1 ROUGE-1 Average_F: 0.38398 (95%-conf.int. 0.38207 - 0.38594)  

---------------------------------------------  
1 ROUGE-2 Average_R: 0.13426 (95%-conf.int. 0.13267 - 0.13581)  
1 ROUGE-2 Average_P: 0.22805 (95%-conf.int. 0.22538 - 0.23062)  
1 ROUGE-2 Average_F: 0.16363 (95%-conf.int. 0.16173 - 0.16540)  

---------------------------------------------  
1 ROUGE-3 Average_R: 0.07596 (95%-conf.int. 0.07452 - 0.07735)  
1 ROUGE-3 Average_P: 0.12769 (95%-conf.int. 0.12541 - 0.13003)  
1 ROUGE-3 Average_F: 0.09206 (95%-conf.int. 0.09045 - 0.09366)  

---------------------------------------------  
1 ROUGE-4 Average_R: 0.04972 (95%-conf.int. 0.04841 - 0.05096)  
1 ROUGE-4 Average_P: 0.08276 (95%-conf.int. 0.08070 - 0.08482)  
1 ROUGE-4 Average_F: 0.05991 (95%-conf.int. 0.05840 - 0.06139)  

---------------------------------------------  
1 ROUGE-L Average_R: 0.24827 (95%-conf.int. 0.24659 - 0.25000)  
1 ROUGE-L Average_P: 0.42276 (95%-conf.int. 0.42031 - 0.42537)  
1 ROUGE-L Average_F: 0.30315 (95%-conf.int. 0.30142 - 0.30493)  

---------------------------------------------  
1 ROUGE-W-1.2 Average_R: 0.07426 (95%-conf.int. 0.07370 - 0.07482)  
1 ROUGE-W-1.2 Average_P: 0.30838 (95%-conf.int. 0.30645 - 0.31043)  
1 ROUGE-W-1.2 Average_F: 0.11689 (95%-conf.int. 0.11610 - 0.11771)  

---------------------------------------------  
1 ROUGE-S* Average_R: 0.09381 (95%-conf.int. 0.09251 - 0.09503)  
1 ROUGE-S* Average_P: 0.25627 (95%-conf.int. 0.25362 - 0.25901)  
1 ROUGE-S* Average_F: 0.12408 (95%-conf.int. 0.12276 - 0.12535)  

---------------------------------------------  
1 ROUGE-SU* Average_R: 0.09889 (95%-conf.int. 0.09757 - 0.10012)  
1 ROUGE-SU* Average_P: 0.26783 (95%-conf.int. 0.26517 - 0.27058)  
1 ROUGE-SU* Average_F: 0.13084 (95%-conf.int. 0.12948 - 0.13213)